# Whisper ASR Fine-Tuning Pipeline for Punjabi (Indic-ASR)

In [1]:
# 📦 1. Imports
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, Audio
import torch
import torchaudio
import evaluate
import numpy as np
import os

In [2]:
# 🧠 2. Environment and GPU Check
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

PyTorch version: 2.7.1+cu118
CUDA available: True
Device: cuda


In [3]:
from datasets import load_dataset, Audio

# Load Common Voice Punjabi from local directory
dataset = load_dataset(
    "mozilla-foundation/common_voice_11_0",
    "pa",
    data_dir="./commonvoice_punjabi",
    split="train[:1%]",
    trust_remote_code=True  # <-- this fixes it
)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
dataset = dataset.rename_column("sentence", "text")
print("✅ Common Voice Punjabi loaded successfully")


ValueError: BuilderConfig 'pa' not found. Available: ['en', 'fa', 'fr', 'es', 'sl', 'kab', 'cy', 'ca', 'de', 'tt', 'ta', 'ru', 'nl', 'it', 'eu', 'tr', 'ar', 'zh-TW', 'br', 'pt', 'eo', 'zh-CN', 'id', 'ia', 'lv', 'ja', 'rw', 'sv-SE', 'cnh', 'et', 'ky', 'ro', 'hsb', 'el', 'cs', 'pl', 'rm-sursilv', 'rm-vallader', 'mn', 'zh-HK', 'ab', 'cv', 'uk', 'mt', 'as', 'ka', 'fy-NL', 'dv', 'pa-IN', 'vi', 'or', 'ga-IE', 'fi', 'hu', 'th', 'lt', 'lg', 'hi', 'bas', 'sk', 'kmr', 'bg', 'kk', 'ba', 'gl', 'ug', 'hy-AM', 'be', 'ur', 'gn', 'sr', 'uz', 'mr', 'da', 'myv', 'nn-NO', 'ha', 'ckb', 'ml', 'mdf', 'sw', 'sat', 'tig', 'ig', 'nan-tw', 'mhr', 'bn', 'tok', 'yue', 'sah', 'mk', 'sc', 'skr', 'ti', 'mrj', 'tw', 'vot', 'az', 'ast', 'ne-NP']

In [ ]:
# 🧩 4. Load Whisper Model and Processor
try:
    processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
    model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="punjabi", task="transcribe")
    model.config.suppress_tokens = []
    print("✅ Whisper model and processor loaded")
except Exception as e:
    print("❌ Failed to load model:", str(e))
    raise

In [ ]:
# 🧼 5. Preprocess Dataset
def prepare_dataset(batch):
    try:
        audio = batch["audio"]
        inputs = processor(audio["array"], sampling_rate=16000)
        batch["input_features"] = inputs.input_features[0]
        with processor.as_target_processor():
            batch["labels"] = processor(batch["text"]).input_ids
    except Exception as e:
        print("Preprocessing error:", str(e))
        batch["input_features"] = []
        batch["labels"] = []
    return batch

In [ ]:
try:
    dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)
    print("✅ Preprocessing completed")
except Exception as e:
    print("❌ Preprocessing failed:", str(e))
    raise

In [ ]:
# ⚙️ 6. Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-punjabi-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    warmup_steps=100,
    max_steps=200,
    save_steps=100,
    eval_steps=100,
    logging_steps=50,
    evaluation_strategy="steps",
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
)

In [ ]:
# 📦 7. Data Collator
def data_collator(features):
    input_features = [{"input_features": f["input_features"]} for f in features if f["input_features"]]
    label_features = [f["labels"] for f in features if f["labels"]]
    batch = processor.feature_extractor.pad(input_features, return_tensors="pt")
    labels_batch = processor.tokenizer.pad({"input_ids": label_features}, return_tensors="pt")
    labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
    batch["labels"] = labels
    return batch

In [ ]:
# 📏 8. Metric Calculation (WER)
metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)
    wer_score = metric.compute(predictions=pred_str, references=label_str)
    print(f"WER: {wer_score:.4f}")
    return {"wer": wer_score}

In [ ]:
# 🏋️‍♂️ 9. Trainer Initialization
try:
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=dataset,
        eval_dataset=dataset,
        tokenizer=processor.feature_extractor,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    print("✅ Trainer initialized")
except Exception as e:
    print("❌ Failed to initialize trainer:", str(e))
    raise

In [ ]:
# 🚀 10. Fine-Tuning the Model
try:
    trainer.train()
    print("✅ Training completed")
except Exception as e:
    print("❌ Training failed:", str(e))
    raise

In [ ]:
# 🧪 11. Evaluate Final WER on a Few Samples
results = []
for example in dataset.select(range(5)):
    try:
        input_data = processor(example["input_features"], return_tensors="pt").to(model.device)
        with torch.no_grad():
            generated_ids = model.generate(input_data["input_features"])
        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        results.append((transcription, example["text"]))
        print("Ref:", example["text"])
        print("Hyp:", transcription)
        print("---")
    except Exception as e:
        print("❌ Inference failed:", str(e))

In [ ]:
results = []
for example in dataset.select(range(5)):
    try:
        input_data = processor(example["input_features"], return_tensors="pt").to(model.device)
        with torch.no_grad():
            generated_ids = model.generate(input_data["input_features"])
        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        results.append((transcription.strip(), example["text"].strip()))
        print("Ref:", example["text"])
        print("Hyp:", transcription)
        print("---")
    except Exception as e:
        print("❌ Inference failed:", str(e))

preds, refs = zip(*results)

# Compute WER
final_wer = metric.compute(predictions=preds, references=refs)
print("🔍 Final WER on samples:", final_wer)

# Compute Accuracy
correct = sum(p.lower() == r.lower() for p, r in zip(preds, refs))
accuracy = correct / len(preds)
print("✅ Accuracy on samples:", accuracy)
